In [ ]:
import os
import sys

sys.path.append(os.path.join('..'))
sys.path.append(os.path.join('..', 'src'))
sys.path.append(os.path.join('..', 'src', 'libs'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LogNorm
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from copy import deepcopy
import seaborn as sns

import hera
from libs import parser as parse
from libs import titration as titi
from libs import util as utl

from ami_utils import parse as pa

import matplotlib as mpl
mpl.rcParams['font.family'] = 'Arial'

#### Generate titration params

In [ ]:
exp_names = ['LUB096', 'LUB141']
# dict_exp_complex = dict(zip(exp_names, ['EH-MebipZn(OTf2)2']))
hera.initialize_titrations(exp_names)

dict_titrants_exps = hera.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(hera.dict_titrations_exps)

dict_titrants_exps

In [ ]:
dict_titrants_exps = hera.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(hera.dict_titrations_exps)
dict_titrants_exps

In [ ]:
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

In [ ]:
dict_titrations_exps['LUB096'][0][0].df_params.host_conc.iloc[0]*1e6

In [ ]:
#TODO: Check parser get the first line of text file.

exps_reps = parse.get_exp_reps(exp_names)
data = pa.get_data(pa.path_constructor('uv', *exps_reps), 'uv')
# data = parse.get_data(exps_reps)
data

In [ ]:
# lub014_df = data.get('LUB014').drop(columns=['70', 'glasspip'])
data = data[~data.titrant.str.contains('mecn')]
data

In [ ]:
new_data = utl.merge_data_titration_params(data, dict_titrations_exps)
new_data

In [ ]:
mebip_df = new_data[new_data.name == 'LUB096']
mbp_df = new_data[new_data.name == 'LUB141']

print(mebip_df.loc[mebip_df.loc[mebip_df['Wavelength nm.'] >= 250, 'Abs.'].idxmax(), 'Wavelength nm.'])
print(mbp_df.loc[mbp_df.loc[mbp_df['Wavelength nm.'] >= 250, 'Abs.'].idxmax(), 'Wavelength nm.'])

#### Generate peak tracking data

In [ ]:
dict_df = utl.tracking_df(new_data, ['max'], exps_reps, complex_tit=False)
dict_df

In [ ]:
df = utl.combine_track_data(new_data, dict_df)
df

In [ ]:
df.loc[df.name == 'LUB141'].g_h.unique()[[10]]

### Visualise data

#### Acid Relationship

##### Line graph

In [ ]:
# Legend used for graph - super manuel

legend_labels = ['EH-MEBIP', 'EH-MBP']

# Designate grouper (to group titrations according to host or guest)
grouper = 'host_name'

In [ ]:
print(sns.color_palette("muted").as_hex())

In [ ]:
print(sns.color_palette("deep").as_hex())

In [ ]:
# Plot in seperate figures
%matplotlib qt
# BG_WHITE = "#fafaf5"
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):
    
    # Set colors for graph
    PalSet = sns.color_palette('muted', len(df.name.unique()))
    m_colors = PalSet.as_hex()
    m_colors = ['#439088']
    
    # Plot data of interest
    to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'value', 'rank']].drop_duplicates()
    to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

    for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
        
        fig, ax = plt.subplots(1) # Set a size once deteremined !
        
        # Background colours
        # fig.patch.set_facecolor(BG_WHITE)
        # ax.set_facecolor(BG_WHITE)
        
        # Border and grid params
        ax.spines[['right', 'top']].set_visible(False)
        
        # axis params
        ax.set_xlabel(f'[HCl]:[{group[grouper].unique()[0]}]')
        ax.set_ylabel(r'${\Delta}A_{\pi\rightarrow\pi^{*}}$')
        ax.xaxis.set_major_locator(MultipleLocator(0.5))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        
        
        # data and seaborn plot
        group.value = (group.value - group.value.min())/(group.value.max() - group.value.min())
        # lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', ax=ax, palette='deep', legend=False)
        lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', ax=ax, palette=m_colors, legend=False)
        
        # Legend - manual if legend required :(
        # new_title = 'Ligand'
        # ax.legend(edgecolor='none')
        # lnplt.legend_.set_title(new_title)
        
        # # Replace labels
        # for text, label in zip(lnplt.legend_.texts, legend_labels):
        #     print(label)
        #     text.set_text(f'{group[grouper].unique()[0].upper()}')
        
        # Scatter plot for hollow spheres - matplotlib
        for (exp, group), col in zip(group.groupby(['name', 'guest_name', 'rank'], as_index=False), m_colors):
            group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
            ax.plot(group_mean.g_h, group_mean.value, 'o', markerfacecolor='None', markeredgewidth=2, markersize=9, markeredgecolor='#55a868', 
                    alpha=0.7, label=exp, lw=2)
            
            # Plot the infliction points as red dots
            # red_dot = group_mean.iloc[[8, 21]]
            red_dot = group_mean.iloc[[10]]
            ax.plot(red_dot.g_h.values, red_dot.value.values ,'o', markerfacecolor='None', markeredgewidth=2, markersize=9, markeredgecolor='#e8000b',
                    alpha=1)

        fig.tight_layout()

##### Spectra

In [ ]:
%matplotlib qt
# Plots each titration individually
# Arange data and group plus plot

# TODO: Change pallete, remove legend
to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

with sns.plotting_context("paper", font_scale=1.75,
                          rc={
                              "lines.linewidth": 1.75, 'lines.markersize': 6.5, 'patch.linewidth': 1.35
                          }):

    for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
        fig, ax = plt.subplots(1) # Set a size once deteremined !
        print(group.loc[group['Wavelength nm.'].between(250, 425)])

        # Background colours
        # BG_WHITE = "#fafaf5"
        # fig.patch.set_facecolor(BG_WHITE)
        # ax.set_facecolor(BG_WHITE)
        
        # Border and grid params
        ax.spines[['right', 'top']].set_visible(False)
        # ax.spines[['bottom']].set_linewidth(2)
        # ax.set_axisbelow(True)
        # ax.grid(axis='y', linestyle=':', lw=1, color='grey')
    
        # axis params
        ax.set_xlabel('Wavelength (nm)')
        ax.set_ylabel('Absorbance (a.u.)')
        # ax.tick_params(length=4, axis='x')
        # ax.tick_params(axis='y',length=0)
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        
        # Plot data
        sns.lineplot(data=group.loc[group['Wavelength nm.'].between(250, 425)], x="Wavelength nm.", y="Abs.", hue='g_h', palette='flare', 
                     legend=False, ax=ax)

        # Colourbar 
        norm = plt.Normalize(group.g_h.min(), group.g_h.max())
        sm = plt.cm.ScalarMappable(cmap='flare', norm=norm)
        cbar = fig.colorbar(sm, shrink=0.7, cax=ax.inset_axes((0.85, 0.125, 0.025, 0.75)))
        cbar.ax.set_title('HCl eq.')
        
        fig.tight_layout()

### Animations

In [ ]:
import matplotlib.animation as animation

In [ ]:
%matplotlib qt
fig, ax = plt.subplots()


x = np.linspace(-5, 5, 100)
y = 3*np.sin(x)

ax.set(xlim=[-5-0.1, 5+0.1], ylim=[yvals.min()-0.2, yvals.max()+0.2])
line, = ax.plot([], [], 'k')
scatter, = ax.plot([], [], 'ko')
print(line)
print(scatter)

def update(frame):
    line.set_data(x[:frame], y[:frame])
    scatter.set_data(x[:frame], y[:frame])

    return (line, scatter)

ani = animation.FuncAnimation(fig=fig, func=update, frames=len(x)+1, interval=80, blit=True)
# ani.save(filename="ffmpeg_example.mp4", writer="ffmpeg")
plt.show()

In [ ]:
# Plot data of interest
to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'value', 'rank']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

In [ ]:
# Plot in seperate figures
%matplotlib qt
# BG_WHITE = "#fafaf5"
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):
    
    # Set colors for graph
    PalSet = sns.color_palette('muted', len(df.name.unique()))
    m_colors = PalSet.as_hex()

    for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
        
        fig, ax = plt.subplots(1) # Set a size once deteremined !
        
        # Border and grid params
        ax.spines[['right', 'top']].set_visible(False)
        
        # axis params
        ax.set_xlabel(f'[HCl]:[{group[grouper].unique()[0]}]')
        ax.set_ylabel(r'${\Delta}A_{\pi\rightarrow\pi^{*}}$')
        ax.xaxis.set_major_locator(MultipleLocator(0.5))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))

        # Norm data
        group.value = (group.value - group.value.min())/(group.value.max() - group.value.min())
        for (exp, group), col in zip(group.groupby(['name', 'guest_name', 'rank'], as_index=False), m_colors):
            group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
            x, y = [group_mean.g_h.values, group_mean.value.values]
            red_dots = group_mean.iloc[[8, 21]].values
            print(red_dots)
        
            print(x)
            ax.set(xlim=[x.min()-0.1, x.max()+0.1], ylim=[y.min()-0.2, y.max()+0.2])
            # line, = ax.plot([], [], 'k')
            scatter, = ax.plot([], [], 'o', markerfacecolor='None', markeredgewidth=2, markersize=9, markeredgecolor=col, 
                              alpha=0.7, lw=2)
            scatter2, = ax.plot([], [] ,'o', markerfacecolor='None', markeredgewidth=2, markersize=9, markeredgecolor='#e8000b',
                                alpha=1)
    
            
            # lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', ax=ax, palette='deep', legend=False)
    
            def update(frame):
                line.set_data(x[:frame], y[:frame])
                scatter.set_data(x[:frame], y[:frame])
                if frame >= 8:
                    scatter2.set_data(x[8], y[8])
                if frame >= 21:
                    scatter2.set_data(x[21], y[21])
    
                return (line, scatter, scatter2)
            
            print(len(x))
            ani = animation.FuncAnimation(fig=fig, func=update, frames=len(x)+1, interval=80, blit=True)
            # ani.save(filename="ffmpeg_example.mp4", writer="ffmpeg")
            plt.show()

        fig.tight_layout()

In [ ]:
%matplotlib qt
# run the animation
ani = animation.FuncAnimation(fig, animate, frames=20, interval=500, repeat=False)
ani.save(filename="ffmpeg_example.mp4", writer="ffmpeg")

plt.show()

### Signal decomposition

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_lub096 = df.pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.').loc[250:]['LUB096']
df_lub141 = df.pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.').loc[250:]['LUB141']

# X = df_lub096.values.T
X = df_lub141.values.T

#### PCA

In [ ]:
# Perform PCA
pca = PCA(n_components=1)
pca.fit(X)
X_red = pca.transform(X)
X_reconstruct = pca.inverse_transform(X_red)

print(f'Explained variance:\n{pca.explained_variance_ratio_}')

In [ ]:
# Plot results

plt.plot(df_lub141.index, pca.components_[:5].T)
plt.title('Components')
plt.legend(['PC1', 'PC2'])
plt.xlabel('Wavelength')
plt.ylabel('Absorbance')
plt.figure()
plt.title('Components_Transformed')
plt.ylabel('Absorbance')
plt.xlabel('Time (min)')
plt.plot(df_lub141.columns.astype(float), X_red, '.')
plt.legend(['PC1', 'PC2'])
plt.figure()
plt.title(f'Reconstruction from first PCA (97 % varaince explained)')
plt.plot(df_lub141.index, X_reconstruct.T)

#### NMF

In [ ]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# MinMax to avoid negative values

X_mm = MinMaxScaler().fit_transform(X.T).T

# Perfrom NMF
nmf  = NMF(n_components=1)
nmf.fit(X_mm)
W = nmf.transform(X_mm)
H = nmf.components_
X_reconstruct = nmf.inverse_transform(W)

In [ ]:
nmf.reconstruction_err_

In [ ]:
# Plot results

plt.plot(df_lub141.index, H.T)
plt.title('Components')
plt.legend(['1', '2'])
plt.xlabel('Wavelegnth')
plt.ylabel('Absorbance')
plt.figure()
plt.plot(df_lub141.columns.astype(float), W, '.')
plt.title('Components_Transformed')
plt.ylabel('Absorbance')
plt.xlabel('Time (min)')
plt.legend(['1', '2'])
plt.figure()
plt.title(f'Reconstruction from two NMF components (0.98 reconstruction error)')
plt.plot(df_lub141.index, X_reconstruct.T)